In [1]:
import os
import requests
import os
import sys
from tqdm import tqdm
import time
import Cobol2Spark as chat
import numpy as np

In [2]:
with open('DWSD0612_completo_V3.txt', 'r') as file:
    data = file.read().strip()
    data = ' '.join(data.split())

In [3]:
final = []
divisions = ["ENVIRONMENT DIVISION", "DATA DIVISION","PROCEDURE DIVISION"]
data_split = data.split('DIVISION')[2:]
for i,row in enumerate(data_split):
    final.append(f'{divisions[i]} {row}')


In [4]:
usage = []
responses = []
for code in tqdm(final):
    response = chat.generate(prompt=f"""Given the following code snippet. Your role will be to decipher the code, create an  PySpark code with the same functionality, including all the necessary processes.

code:
```
{code}
```

Your task is to do the following steps:

1. Analyze and comprehend the COBOL code snippet above.
2. Write an equivalent PySpark code.
3. The equivalent code should provide the exact same output as the COBOL code.
4. Pay extra attention in the every data processing steps like filtering, aggregation, and sorting.
5. Follow best practices while crafting your code, like writing clean, well-indented, efficient, and concise code.
6. Return only the PySpark code.
""",temperature=0.4, max_tokens=1000)
    responses.append(response)
    usage.append(((response.usage.prompt_tokens/1000) * 0.006)+((response.usage.completion_tokens/1000) * 0.012))
print(np.sum(usage))

100%|██████████| 3/3 [02:59<00:00, 59.98s/it]

0.15576600000000002


In [5]:
final_sparks = '\n\n'.join(resp.choices[0].message.content for resp in responses)

In [8]:
print(final_sparks)

from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder \
    .appName('Translate COBOL to PySpark') \
    .getOrCreate()

# Read data from files into DataFrames
df_arqparm = spark.read.format('csv').option('header', 'true').load('ARQPARM.csv')
df_arqsort = spark.read.format('csv').option('header', 'true').load('ARQSORT.csv')
df_dimestip = spark.read.format('csv').option('header', 'true').load('DIMESTIP.csv')
df_arqftp = spark.read.format('csv').option('header', 'true').load('ARQFTP.csv')


df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').option('delimiter', ';').option('decimal', ',').load('file.csv')



 from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the data into a DataFrame
df = spark.read.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="ATSAUDAO.SSCESTIPULANTE",
    user="your_username",
    password="your_password").loa

In [7]:
import re

sparks = []
for resp in  responses:
    code = '\n\n'.join(re.findall(r'```python\n(.*?)```', resp.choices[0].message.content, re.DOTALL))
    sparks.append(code) 
final_sparks = '\n\n\n '.join(sparks)

In [ ]:
cobol_final = '\n\n'.join(final)

In [ ]:
final_code = chat.generate(prompt=f"""Analise the following codes snippet and decipher the code.
                            

cobol code:
```
{cobol_final}
```

Pyspark code:
```
{final_sparks}
```

Your task is to:

1. Analyze and comprehend the PySpark and cobol code snippet above.
2. verify if in the Spark code there is any mistake or if it is not equivalent to the COBOL code.
3. If there is any mistake, correct it.
4. If the code is missing some functionality, add it.
5. pay extra attention to the data types and the every data processing steps like filtering, aggregation, and sorting.
6. Return only the PySpark code.""",temperature=0.4,max_tokens=1000)

print(((final_code.usage.prompt_tokens/1000) * 0.006)+((final_code.usage.completion_tokens/1000) * 0.012))

In [ ]:
print(final_code.choices[0].message.content)